<a href="https://colab.research.google.com/github/karim-mammadov/Computer-Vision-My-projects/blob/main/Animal_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/animal-faces

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/animal-faces.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
data_path="/content/afhq"

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
data_dir = '/content/afhq/train'
categories = os.listdir(data_dir)

In [ ]:
# sinif pay dağılımı
import os, pandas as pd

# Fayl yollarının və etiketlərinin siyahısını yaratmaq
filepaths = []
labels = []

for category in categories:
    category_path = os.path.join(data_dir, category)
    for filename in os.listdir(category_path):
        filepaths.append(os.path.join(category_path, filename))
        labels.append(category)

# datanı yaratmaq
df = pd.DataFrame({'filepath': filepaths, 'label': labels})


# Hər bir kateqoriya üçün saylarını hesablamaq
counts = df['label'].value_counts()
counts = counts.reindex(['cat', 'dog', 'wild']) # sifarişin indeksə uyğun olduğundan əmin olmaq

counts.plot(kind='bar')
plt.show()


# nümunə şəbəkəsi
fig, axes = plt.subplots(3,5, figsize=(10,6))
for i, cls in enumerate(['cat','dog','wild']):
    sel = df[df['label']==cls].sample(5)
    for j, row in enumerate(sel.itertuples()):
        img = plt.imread(row.filepath)
        axes[i,j].imshow(img); axes[i,j].axis('off')

plt.show()

In [ ]:
# Hər kateqoriyadan bir şəkil seçmək
for category in categories:
    image_path = os.path.join(data_dir, category, os.listdir(os.path.join(data_dir, category))[0])
    img = mpimg.imread(image_path)
    plt.figure(figsize=(3, 3))
    plt.imshow(img)
    plt.title(category)
    plt.axis('off')
    plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np

# img_height = 224
# img_width = 224
batch_size = 32
img_size=(224,224)

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_dir='/content/afhq/train'
val_dir='/content/afhq/val'

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(kernel_size=(7,7), filters=32, strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(kernel_size=(5,5), filters=64, strides=2, activation='relu', kernel_initializer='he_normal'),  #(7,7)
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, strides=2, activation='relu', kernel_initializer='he_normal'),  # 5,5
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),  #0,5
    tf.keras.layers.Dense(units=128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=3, activation='softmax')

])

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.AdamW(),  #adam
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=10,
    steps_per_epoch=len(train_ds),
    validation_data=val_ds,
    validation_steps=len(val_ds)
)

In [ ]:
# Vizual ilə overfitting və underfittingin olub olmadigini təyin edirik
epochs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
train_acc = [0.6578, 0.8863, 0.9298, 0.9461, 0.9498, 0.9669, 0.9697, 0.9732, 0.9777, 0.9861]
val_acc   = [0.7820, 0.8420, 0.8773, 0.9507, 0.8800, 0.9320, 0.9140, 0.9560, 0.9173, 0.9580]


plt.plot(epochs, train_acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Validation loss bəzən artır bu da potensial overfitting siqnalı verə bilər. Son epoch-da yaxşılaşma var (Nəticə: 0.1476).
train_loss = [0.7809, 0.2972, 0.1907, 0.1506, 0.1368, 0.0903, 0.0868, 0.0794, 0.0612, 0.0388]
val_loss   = [0.5576, 0.4066, 0.4127, 0.1586, 0.4399, 0.2060, 0.2836, 0.1699, 0.2911, 0.1476]

plt.plot(epochs, train_loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train vs Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# @title
